In [ ]:
import numpy as np
import re
import json

import pandas as pd
import numpy as np
import re
import os

def clean(n):
    n = ''.join(n.split())
    n = n.replace('n|','').replace('ic-','').replace('stx-','').replace('[KoG]^','').replace(']km[','')
    n = n.replace('[B]','').replace('territory.ru>','').replace('esp-','').replace('ss-','').replace('K9-','')
    n = n.replace('socrates_','socrates').replace('ASUS*c58-','').replace('ve-','').replace('g1|','')
    n = n.replace('TOXIC','toxjq').replace('.','').replace('territoryru`','').replace('fazzz','fazz')
    if n[-1] == '_':
        n = n[:-1]
    return n
def clean2(x):
    for sub in ['cK-','All*','{DigA}','c4','VP.','myztro|','CG_','POMPA','M_','myztro','__','SK','(invitationpending)','/CV']:
        x = x.replace(sub,'')
    return x

In [ ]:
brack_text = {}
for t in os.listdir('brackets'):
    brack_text[t] = open('brackets/' + t).read()
    
import pickle
with open('flags.pkl','rb') as fp:
    flags = pickle.load(fp)
flags['pow3r'] = 'ru'
flags['slinger'] = 'de'
flags['treble'] = 'se'
flags['zyz'] = 'de'
flags['polosatiy'] = 'ru'
flags['powerk'] = 'kr'
flags['neok'] = 'de'
flags['shurik'] = 'ru'
flags['fazz'] = 'se'
flags['ztalwart'] = 'se'
def get_codes(example_str):
    codes = []
    for line in example_str.split('\n'):
        codes.append(line[1:].strip().rstrip().split(' |')[0][:-1])

    codes = [_ for _ in codes if len(_) > 0]
    return codes
brack_codes = {k: get_codes(v) for k,v in brack_text.items()}


In [ ]:
data = json.load(open('fps88.json',encoding='utf-8'))
mbr = data['matches_by_round']

In [ ]:
[(k,len(v)) for k,v in mbr.items()]

In [ ]:
fr_u = -max([(len(v),-int(k)) for k,v in mbr.items() if '-' not in k])[1]
fr_l = max([(len(v),int(k)) for k,v in mbr.items() if '-' in k] )[1]

In [ ]:
i = fr_u
matches_upper = []
while True:
    if str(i) not in mbr:
        break
    round_m = []
    for match in mbr[str(i)]:
        p1 = clean2(clean(clean2(match['player1']['display_name'])))
        p2 = clean2(clean(clean2(match['player2']['display_name'])))
        try:
            s1, s2 = match['scores']
        except:
            s1 = match['winner_id'] == match['player1']['id']
            s2 = match['winner_id'] == match['player2']['id']

        s1,s2 = int(s1),int(s2)
        round_m.append((p1,p2,s1,s2))
        
    matches_upper.append(round_m)
    i+=1

i = fr_l
matches_lower = []
while True:
    if str(i) not in mbr:
        break
    round_m = []
    for match in mbr[str(i)]:
        p1 = clean2(clean(clean2(match['player1']['display_name'])))
        p2 = clean2(clean(clean2(match['player2']['display_name'])))
        try:
            s1, s2 = match['scores']
        except:
            s1 = match['winner_id'] == match['player1']['id']
            s2 = match['winner_id'] == match['player2']['id']

        s1,s2 = int(s1),int(s2)
        round_m.append((p1,p2,s1,s2))
    i-=1
    matches_lower.append(round_m)
print(matches_upper[-1])
print(matches_lower[-1])
matches_upper = matches_upper[1:-1]
matches_lower = matches_lower[1:-1]

[len(_) for _ in matches_upper],[len(_) for _ in matches_lower]

In [ ]:
matches_lower

In [ ]:
TYPE = 'se' if len(matches_lower) == 0 else 'de'
codes = brack_codes[str(2*len(matches_upper[0])) + TYPE + '.txt']

In [ ]:
tres = []
for brack in [_ for _ in [matches_upper,matches_lower] if len(_) > 0]:
    res = []

    queue = [list(_) + [0] for _ in brack[-1]]
    all_match_us = sum(brack[:-1],[])[::-1]

    temp_res = []
    curr_level = 0
    print(queue,res)
    while len(queue) > 0:
        print(len(queue),len(res),curr_level, len(temp_res),len(all_match_us))

        match=queue.pop(0)
        flip = False
        if match[-1] != curr_level:
            curr_level += 1
            res.append(temp_res)
            temp_res = []
        del_idx = -1
        for i in range(len(all_match_us)):
            if all_match_us[i][0] == match[0] or all_match_us[i][1] == match[0]:
                queue.append(list(all_match_us[i]) + [curr_level+1])
                del_idx = i
                break
        #print(del_idx,len(all_match_us))
        if del_idx != -1:
            del all_match_us[del_idx]
            flip =True
        del_idx = -1
        for i in range(len(all_match_us)):
            if all_match_us[i][0] == match[1] or all_match_us[i][1] == match[1]:
                queue.append(list(all_match_us[i]) + [curr_level+1])
                del_idx = i
                break
        #print(del_idx,len(all_match_us))
        if del_idx != -1:
            del all_match_us[del_idx]
            flip = False
        if flip:
            match = (match[1],match[0],match[3],match[2],match[4])
        temp_res.append(match)
    #temp_res.append(['schnucks', 'SoulXtort', 0, 0] + [curr_level])
    res.append(temp_res)
    tres.append(res)

In [ ]:
[[len(x) for x in _ ] for _ in tres]

In [ ]:
tres

In [ ]:
TYPE

In [ ]:
new_res = sum([_[::-1] for _ in tres],[])
new_res = sum(new_res,[])
#|R1D43=belmakor|R1D43score=15 |R1D43flag= |R1D43win=1
#|R1D44=dCypher|R1D44score=7 |R1D44flag= |R1D44win=
for i in range(0,len(new_res)):
    p1,p2,s1,s2,lvl = new_res[i]
    s1,s2 = int(s1),int(s2)
    w1 = '1' if int(s1) > int(s2) else ''
    w2 = '1' if int(s2) > int(s1) else ''
    bn =codes[2*i]
    if abs(s1) + abs(s2) ==1:
        s1 = ''
        s2 = ''
    f1 = flags.get(p1.lower(),'')
    f2 = flags.get(p2.lower(),'')

    print('|{}={}|{}score={} |{}flag={} |{}win={}'.format(bn,p1,bn,s1,bn,f1,bn,w1))
    bn = codes[2*i+1]
    print('|{}={}|{}score={} |{}flag={} |{}win={}'.format(bn,p2,bn,s2,bn,f2,bn,w2))
print()
for i in range(i+1,len(codes)//2):
    p1,p2 = '',''
    s1,s2 ='',''
    w1 = ''
    w2 = ''
    bn =codes[2*i]
    f1 = ''#flags.get(p1.lower(),'ru')
    f2 = ''
    print('|{}={}|{}score={} |{}flag={} |{}win={}'.format(bn,p1,bn,s1,bn,f1,bn,w1))
    bn = codes[2*i+1]
    print('|{}={}|{}score={} |{}flag={} |{}win={}'.format(bn,p2,bn,s2,bn,f2,bn,w2))


In [ ]:
matches_upper[0]

In [ ]:
codes